In [35]:
### parameters
place = 'tel_aviv'
feature = 'shadows'

In [36]:
import rasterio
from rasterio.features import shapes
import rasterio


import pandas as pd
import geopandas as gpd

from shapely.geometry import LineString, shape,GeometryCollection
from shapely.ops import split
from tqdm import tqdm  # for progress bar





import pickle
import glob
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'


# Get the current working directory (e.g., the folder you're running from)
cwd = Path().resolve()

# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)
os.makedirs(f'{parent_folder}/shp/{feature}',exist_ok=True)
detail_folder = f'{data_folder}/{feature}'

In [13]:
# START FROM THE BEGINNING

# Folder with rasters
raster_folder = glob.glob(f"{detail_folder}/data_source/*.tiff") # Replace with your folder path



# Output list of GeoDataFrames
vector_layers = []

for path  in raster_folder :
    print(f"Processing {path}...")

    with rasterio.open(path) as src:
        image = src.read(1)
        transform = src.transform
        crs = src.crs
        mask = image != src.nodata  # Skip no-data pixels

        polygons = []
        values = []
        # Create mask to ignore nodata values
        for geom, value in shapes(image, mask=mask, transform=transform):
            polygons.append(shape(geom))
            values.append(value)

        # Create GeoDataFrame in original CRS
        gdf = gpd.GeoDataFrame({'value': values, 'geometry': polygons,'file_name':path.split('\\')[-1].split('.')[0]}, crs=crs)

        # Reproject to ITM
        gdf_itm = gdf.to_crs(crs_prj)

        # Optionally save to file
        # output_path = os.path.join(raster_folder, f"{filename}_vector_ITM.geojson")
        # gdf_itm.to_file(output_path, driver='GeoJSON')

        # Store in list
        vector_layers.append(gdf_itm)

print("Finished converting all rasters.")
# Save
with open(f'{detail_folder}/data.pkl', 'wb') as f:
    pickle.dump(vector_layers, f)



Processing C:\Users\18059\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/shadow/data_source\1.tiff...
Processing C:\Users\18059\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/shadow/data_source\10.tiff...
Processing C:\Users\18059\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/shadow/data_source\11.tiff...
Processing C:\Users\18059\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/shadow/data_source\12.tiff...
Processing C:\Users\18059\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/shadow/data_source\13.tiff...
Processing C:\Users\18059\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/shadow/data_source\14.tiff...
Processing C:\Users\18059\OneDrive - ariel.ac.il\Current_research\ASC2\pythonProject/places/tel_aviv/shp/shadow/data_source\15.tiff...
Processing C:\Users\18059\OneDrive - ariel.ac.il\Current

In [37]:
# Load -  IF NEEDED
with open(f'{detail_folder}/data.pkl', 'rb') as f:
    vector_layers = pickle.load(f)

In [38]:
full_gdf = pd.concat(vector_layers, ignore_index=True)
full_gdf.to_file(f'{detail_folder}/all_data.shp')

In [40]:

def check_line_geometry(gdf):
    """
    Ensure all geometries in the GeoDataFrame are LineString.
    """
    if not all(gdf.geometry.type == "LineString"):
        raise ValueError("All geometries must be LineString.")

def get_intersecting_values(segment, polygons):
    """
    Return unique values and matching polygons that intersect with the given segment.
    """
    matches = polygons[polygons.intersects(segment)]
    return matches['value'].unique(), matches

def recursive_split(segment, polygons, min_length=5):
    """
    Recursively split a segment until each part intersects only polygons with a single value,
    or becomes shorter than `min_length`, in which case assign value 0.
    """
    values, _ = get_intersecting_values(segment, polygons)

    if len(values) == 0:
        raise ValueError(f"Street does not intersect any polygon geometry: {segment}")
    if len(values) == 1:
        return [{'geometry': segment, 'value': values[0]}]
    if segment.length < min_length:
        return [{'geometry': segment, 'value': 0}]

    # Split segment in half at midpoint
    mid_point = segment.interpolate(0.5, normalized=True)
    splitter = mid_point.buffer(0.01).boundary
    split_result = split(segment, splitter)

    if isinstance(split_result, GeometryCollection):
        parts = [g for g in split_result.geoms if isinstance(g, LineString)]
    else:
        parts = list(split_result)

    results = []
    for part in parts:
        if part.length > 0:
            results.extend(recursive_split(part, polygons, min_length=min_length))

    return results

def process_streets(streets, shadow_polygons, sindex, min_length=5):
    """
    Process each street:
    - Intersect with shadows polygons.
    - Split if values vary.
    - Return GeoDataFrame of segments with assigned values and lengths.
    """
    check_line_geometry(streets)
    result_rows = []

    for _, row in tqdm(streets.iterrows(), total=len(streets), desc="Processing streets"):
        oid = row['oidrechov']
        line = row.geometry

        candidate_idx = list(sindex.intersection(line.bounds))
        candidates = shadow_polygons.iloc[candidate_idx]
        candidates = candidates[candidates.intersects(line)]

        split_results = recursive_split(line, candidates, min_length=min_length)

        for res in split_results:
            result_rows.append({
                'oidrechov': oid,
                'geometry': res['geometry'],
                'value': res['value'],
                'length': res['geometry'].length
            })

    return gpd.GeoDataFrame(result_rows, crs=streets.crs)

def compute_weighted_average(gdf):
    """
    Compute length-weighted average value per street (by oidrechov).
    """
    return (
        gdf.groupby('oidrechov')
        .apply(lambda x: np.average(x['value'], weights=x['length']))
        .reset_index(name='weighted_shadow')
    )

# === Load Data ===
streets = gpd.read_file(f'{data_folder}/streets.shp')[['oidrechov', 'geometry']]
merged_shadow = full_gdf.copy()
sindex_merged_shadow = merged_shadow.sindex

# === Process ===
segments_gdf = process_streets(streets, merged_shadow, sindex_merged_shadow)

# === Aggregate ===
final_avg = compute_weighted_average(segments_gdf)
output = streets.merge(final_avg, on="oidrechov", how="left")
output['length'] = output.length

# === Export ===
output.to_file(f'{detail_folder}/{feature}.shp')


 91%|█████████ | 7921/8751 [00:01<00:00, 6245.20it/s] 

        value                                           geometry file_name
345610  255.0  POLYGON ((181656.166 670684.137, 181655.837 67...        45
358251  255.0  POLYGON ((179367.476 670118.513, 179362.694 66...         6
352607  255.0  POLYGON ((179443.446 671009.316, 179438.672 66...         5
88142   255.0  POLYGON ((179413.356 671910.761, 179408.578 67...         2
2103    255.0  POLYGON ((179426.821 672701.235, 179422.043 67...         1
311758  255.0  POLYGON ((181011.643 671010.991, 181011.323 67...         4
86975     0.0  POLYGON ((182345.348 671504.72, 182345.341 671...         2
205635  255.0  POLYGON ((181042.97 671767.989, 181042.566 671...         3


ValueError: Assigning CRS to a GeoDataFrame without a geometry column is not supported. Supply geometry using the 'geometry=' keyword argument, or by providing a DataFrame with column name 'geometry'